## Operating a RAG pipeline using locally hosted small language models

A small language model (SLM) in this case is defined as a language model that is less than 10B parameters.  Such models usually take up about 5GB of disk space and can be executed locally on any reasonable spec machine.

In this notebook, we set up the following RAG architecture with the NY Times comment set using a local ChromaDB vector database to handle the vector embeddings of the comment set, and using a few options for SLMs involved using ollama. 

![Simple RAG Architecture](../rag_architecture.jpg) 

Note that to use ollama you need to install it on your machine.  See [the ollama website](https://ollama.com/) for details.  This workflow also assumes that you have already created the Chroma vector database (see the `chromadb_prep` folder for instructions).

In [9]:
# packages
from langchain_community.llms import Ollama
import chromadb
import pandas as pd

# location of pre-built local chromadb
CHROMA_DATA_PATH = "../chroma_data/"
collection_db="article_comments"

# chroma client 
chroma_client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

# SLM options via Ollama
mistral = Ollama(model="mistral")
llama3 = Ollama(model="llama3:8b")
gemma = Ollama(model="gemma")

# helper function for prompt construction
def construct_prompt(docs: dict, question: str) -> str:
    # convert the docs into a numbered list of comments
    results_df = pd.DataFrame(docs['documents']).transpose()
    results_df.columns = ['Comment']
    results_df['ComNum'] = [str(i) for i in range(1, len(results_df) + 1)]
    results_df['Numbered Comments'] = results_df['ComNum'] + '. ' + results_df['Comment']

    # Collect the results in a context
    context = "\n".join([r for r in results_df['Numbered Comments']])

    # construct prompt
    prompt = f"""
        Answer the following question: {question}.  
        Refer only to the following numbered list of comments from NY Times readers when answering: {context}.
        Check each numbered comment very carefully and ignore it if it does not contain language that is a close match to the original question.
        Provide as much information as possible in the summary, subject to the conditions already given.
        Begin your answer with 'Based on the responses from selected NY Times readers', and try to give a sense of majority and minority opinions on the topic, but only if there is an identifiable majority opinion.
        If there is not enough information provided to give a summarized opinion, indicate that this is the case.
        """

    return prompt

# RAG pipeline function
def ask_question_local(question:str, llm: Ollama() = llama3, 
                       collection: chromadb.PersistentClient() = collection_db, n_docs:int = 50, 
                       filters: dict ={}) -> str:
    
    # Find close documents in chromadb
    collection = chroma_client.get_collection(collection)
    results = collection.query(
       query_texts=[question],
       n_results=n_docs,
       where=filters
    )

    prompt = construct_prompt(results, question)
    
    # generate response
    print(llm.invoke(prompt))

In [10]:
# test function
ask_question_local("What do readers think about global warming?", llm = llama3)

Based on the responses from selected NY Times readers, it appears that there is a overwhelming consensus among the respondents that global warming is real and primarily caused by human activities. Only a few comments express skepticism or doubt about the existence of global warming.

The majority of commenters (approximately 40-50) firmly believe that human-caused CO2 emissions are warming the planet at an alarming rate, with many expressing frustration and anger towards those who deny this fact. Some commenters even mention the devastating consequences of climate change, such as rising sea levels, increased natural disasters, and loss of biodiversity.

A smaller minority (around 5-10%) question the severity or cause of global warming. Some argue that there is no consensus among scientists on the issue, while others propose alternative explanations for climate change, such as natural cycles or solar activity. However, these comments are largely outnumbered by those that strongly affirm